# Twitter API ideas :
- why? "always listen to your customer" (what they REALLY saying. which is where they express themselves in twitter & reddit). see sentiment score of given topic & most used keyword. (to understand news on a particular topic fast OR one of feature for stock algo)
- have a frontend that input a query
- returns 2 things:
    1) sentiment score of that given keyword from both reddit AND twitter
    2) wordcount from both reddit AND twitter


- able to identify current trend of the market by listening directly to users on a certain topic sentiments ?

# Obtain
- SEMN steps do later
- codes to retrieve twitter [here](https://github.com/twitterdev/Twitter-API-v2-sample-code)

In [1]:
# reference : https://towardsdatascience.com/searching-for-tweets-with-python-f659144b225f

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()


BEARER_TOKEN = os.getenv('TWITTER_BEARER_TOKEN')
#define search twitter function
def search_twitter(query, tweet_fields, bearer_token = BEARER_TOKEN):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}

    url = "https://api.twitter.com/2/tweets/search/recent?query={}&{}".format(
        query, tweet_fields
    )
    response = requests.request("GET", url, headers=headers)

    print(response.status_code)

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [2]:
#search term
query = "skateboarding dog"
#twitter fields to be returned by api call
tweet_fields = "tweet.fields=text,author_id,created_at"

#twitter api call
json_response = search_twitter(query=query, tweet_fields=tweet_fields, bearer_token=BEARER_TOKEN)
#pretty printing
print(json.dumps(json_response, indent=4, sort_keys=True))

200
{
    "data": [
        {
            "author_id": "1256654802597089282",
            "created_at": "2021-08-06T03:26:24.000Z",
            "id": "1423485582861348865",
            "text": "I MET THE SKATEBOARDING DOG https://t.co/QunznF2cmY"
        },
        {
            "author_id": "474909411",
            "created_at": "2021-08-06T01:02:38.000Z",
            "id": "1423449403671683075",
            "text": "Just in case anyone was keeping track at home, the Olympic skateboarding events were dog shit, and it was probably one of the worst organized events of the games. \n\nParis will come up big time on this one. \n\nCowabunga \ud83e\udd19\ud83c\udffb"
        },
        {
            "author_id": "1023410515",
            "created_at": "2021-08-05T22:25:42.000Z",
            "id": "1423409909643763716",
            "text": "RT @VippusaO: If you\u2019ve already seen a dog skateboarding in overalls today just keep on scrolling...\nvia RecChapman \nhttps://t.co/E8SkBW3pr\u2026"
